In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds

In [2]:
# tfds.load returns a tf.data.Dataset object and a tf.core.DatasetInfo object. The latter contains info about how the set is split 
# into test and train, number of examples and names of features
dataset, metadata = tfds.load('cifar10', as_supervised=True, with_info=True) 
train_dataset, test_dataset = dataset['train'], dataset['test']

In [3]:
# Size of training and test sets
train_size = metadata.splits['train'].num_examples
test_size = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(train_size))
print("Number of test examples:     {}".format(test_size))

Number of training examples: 50000
Number of test examples:     10000


In [4]:
# Normalise data
def minmaxScaling(images, labels):
    images = tf.cast(images, tf.float32)
    images = images/255
    return images, labels

train_dataset = train_dataset.map(minmaxScaling)
test_dataset = test_dataset.map(minmaxScaling)

In [5]:
metadata

tfds.core.DatasetInfo(
    name='cifar10',
    version=1.0.2,
    description='The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.',
    urls=['https://www.cs.toronto.edu/~kriz/cifar.html'],
    features=FeaturesDict({
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple layers of features from tiny images},
        institution = {},
        year = {2009}
    }""",
    redistribution_info=,
)

In [6]:
# Model setup
# One dense layer with 128 neurons, output layer with 10 neurons
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=([32,32,3])),
    tf.keras.layers.Dense(units=128, input_shape=([3072]), activation=('relu')),
    tf.keras.layers.Dense(units=10, input_shape=([1]), activation=('softmax'))
])
# Compile model
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer = 'adam',
              #optimizer = tf.keras.optimizers.Adam(0.1), 
              metrics=['accuracy'])

In [7]:
# Shuffling and splitting of training set 
BATCH_SIZE = 32
train_dataset = train_dataset.repeat().shuffle(train_size).batch(BATCH_SIZE)

In [10]:
# Training model
model.fit(train_dataset, epochs=20, verbose=True, steps_per_epoch=np.ceil(train_size/BATCH_SIZE))

Epoch 1/20
1563/1563 [==============================] - 38s 25ms/step - loss: 1.5909 - accuracy: 0.4301
Epoch 2/20
1563/1563 [==============================] - 28s 18ms/step - loss: 1.5756 - accuracy: 0.4352
Epoch 3/20
1563/1563 [==============================] - 29s 19ms/step - loss: 1.5653 - accuracy: 0.4415
Epoch 4/20
1563/1563 [==============================] - 30s 19ms/step - loss: 1.5544 - accuracy: 0.4423
Epoch 5/20
1563/1563 [==============================] - 30s 19ms/step - loss: 1.5482 - accuracy: 0.4441
Epoch 6/20
1563/1563 [==============================] - 31s 20ms/step - loss: 1.5386 - accuracy: 0.4481
Epoch 7/20
1563/1563 [==============================] - 30s 19ms/step - loss: 1.5240 - accuracy: 0.4567
Epoch 8/20
1563/1563 [==============================] - 31s 20ms/step - loss: 1.5196 - accuracy: 0.4542
Epoch 9/20
1563/1563 [==============================] - 32s 21ms/step - loss: 1.5144 - accuracy: 0.4564
Epoch 10/20
1563/1563 [==============================] - 33s 21m